In [0]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.misc 
import scipy.io
%matplotlib inline  
print ("Packages loaded.")

Packages loaded.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
l = np.load('/content/gdrive/My Drive/custom_data.npz')

print(l.files)

['testlabel', 'categories', 'trainimg', 'trainlabel', 'testimg', 'use_gray', 'imgsize']


In [0]:
# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
use_gray = l['use_gray']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]
print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("Image size is %s" % (imgsize))
print ("%d classes" % (nclass))

print(trainimg)
print(trainlabel)
print(use_gray)

5868 train images loaded
3912 test images loaded
4096 dimensional input
Image size is [64 64]
40 classes
[[0.99215686 0.99607843 0.99607843 ... 0.35294118 0.44313725 0.56078431]
 [0.94117647 0.92156863 0.84705882 ... 0.82352941 0.80784314 0.81568627]
 [1.         1.         1.         ... 0.96078431 0.96078431 0.96470588]
 ...
 [0.61568627 0.63529412 0.65098039 ... 0.78823529 0.77647059 0.79215686]
 [0.75686275 0.71764706 0.68627451 ... 0.56078431 0.58039216 0.60784314]
 [0.80392157 0.90196078 0.78823529 ... 0.5254902  0.5254902  0.59215686]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
1


In [0]:
tf.set_random_seed(0)
n_input  = dim
n_output = nclass

In [0]:
def conv_basic(x):

    x = tf.reshape(x, shape=[-1, imgsize[0], imgsize[1], 1])
    w1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
    conv1 = tf.nn.conv2d(x, w1, [1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    
    w2 = tf.Variable(tf.random_normal([3,3,32,32], stddev=0.01))
    conv2 = tf.nn.conv2d(conv1, w2, [1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

    w3 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
    conv3 = tf.nn.conv2d(conv2, w3, [1,1,1,1], padding='SAME')
    conv3 = tf.nn.relu(conv3)
    conv3 = tf.nn.max_pool(conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    
    w4 = tf.Variable(tf.random_normal([3,3,64,64], stddev=0.01))
    conv4 = tf.nn.conv2d(conv3, w4, [1,1,1,1], padding='SAME')
    conv4 = tf.nn.relu(conv4)
    conv4 = tf.nn.max_pool(conv4, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

    wid = int(conv3.shape[1])
    heg = int(conv3.shape[2])
    
    fc = tf.reshape(conv3, [-1, 64*wid*heg])

    fc_w1 = tf.get_variable("fc_w1", shape=[64*wid*heg, 1024], initializer=tf.contrib.layers.xavier_initializer())
    fc_b1= tf.Variable(tf.random_normal([1024]))
    fc1 = tf.matmul(fc,fc_w1)+fc_b1

    fc_w2 = tf.get_variable("fc_w2", shape=[1024, 40], initializer=tf.contrib.layers.xavier_initializer())
    fc_b2 = tf.Variable(tf.random_normal([40]))
    logits = tf.matmul(fc1,fc_w2)+fc_b2
    
    return logits

print ("NETWORK READY")

NETWORK READY


In [0]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

# Functions! 
_pred = conv_basic(x)            
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=y))
WEIGHT_DECAY_FACTOR = 0.0001
l2_loss = tf.add_n([tf.nn.l2_loss(v) 
            for v in tf.trainable_variables()])
cost = cost + WEIGHT_DECAY_FACTOR*l2_loss
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy

print ("FUNCTIONS READY")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

FUNCTIONS READY


In [0]:
# Parameters
training_epochs = 100
batch_size      = 3
display_step    = 1

# Launch the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain/batch_size)+1
    # Loop over all batches
    for i in range(num_batch): 
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys
                                  })
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys
                                })/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost: %.9f" % 
               (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs
                                , y: batch_ys})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: testimg
                                , y: testlabel})
        print (" Test accuracy: %.3f" % (test_acc))
print ("Optimization Finished!")

Epoch: 000/100 cost: 4.071836535
 Training accuracy: 0.000
 Test accuracy: 0.042
Epoch: 001/100 cost: 3.948389979
 Training accuracy: 0.000
 Test accuracy: 0.030
Epoch: 002/100 cost: 3.875622574
 Training accuracy: 0.000
 Test accuracy: 0.049
Epoch: 003/100 cost: 3.850402835
 Training accuracy: 0.000
 Test accuracy: 0.043
Epoch: 004/100 cost: 3.773463709
 Training accuracy: 0.000
 Test accuracy: 0.030
Epoch: 005/100 cost: 3.724049176
 Training accuracy: 0.000
 Test accuracy: 0.033
Epoch: 006/100 cost: 3.680573809
 Training accuracy: 0.333
 Test accuracy: 0.049
Epoch: 007/100 cost: 3.666787054
 Training accuracy: 0.000
 Test accuracy: 0.049
Epoch: 008/100 cost: 3.647129200
 Training accuracy: 0.333
 Test accuracy: 0.043


KeyboardInterrupt: ignored

In [0]:
sess.close()
print ("Session closed.")